In [58]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sys
from buffer import Buffer

In [59]:
%matplotlib notebook

In [60]:
# Video file info:
# Frame size: 480 x 270
# RGB format (akin to assignments)
# FPS: 30
PATH = "../data/dataset1/Videos/data_test1.rgb"
IMG_WIDTH = 480
IMG_HEIGHT = 270
IMG_DEPTH = 3

In [12]:
f = open(PATH, 'rb')

In [19]:
# Read first image in file
bit_str = f.read(IMG_WIDTH * IMG_HEIGHT * IMG_DEPTH)
np_arr = np.frombuffer(bit_str, np.uint8)

In [20]:
# Display image
img = np.zeros((IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH), dtype=np.uint8)

for y in range(IMG_HEIGHT):
    for x in range(IMG_WIDTH):
        idx = y * IMG_WIDTH + x
        for d in range(IMG_DEPTH):
            #img[y, x, d] = np_arr[idx + d] <--- this approach does not work
            img[y, x, d] = np_arr[idx + IMG_WIDTH * IMG_HEIGHT * d] #<--- this is how its done in the hw1 java file
            
# This is a necessary step because OpenCV does everything in BGR
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            
cv2.imshow("", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
# buffer example

b = Buffer(5)
b.add(1)
b.add(2)
b.add(3)
b.add(4)
b.add(5)
print(b.buffer)
b.add(6)
print(b.buffer)

In [61]:
f = open(PATH, 'rb')
f_size = IMG_WIDTH * IMG_HEIGHT * IMG_DEPTH

bit_str = f.read(f_size)
bit_arr = np.frombuffer(bit_str, np.uint8)
img = np.zeros((IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH), dtype=np.uint8)
for d in range(IMG_DEPTH):
    channel = np.reshape(bit_arr[d*IMG_WIDTH*IMG_HEIGHT:(d+1)*IMG_WIDTH*IMG_HEIGHT], (IMG_HEIGHT, IMG_WIDTH))
    img[:,:,d] = channel
#img = np.reshape(bit_arr, (IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH), order='A')

In [41]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

In [62]:
w = cv2.VideoWriter()

### timing experiments

In [48]:
f = open(PATH, 'rb')
f_size = IMG_WIDTH * IMG_HEIGHT * IMG_DEPTH

In [49]:
%%timeit

# Operate on input with reshape rather than triple for-loop

bit_str = f.read(f_size)
bit_arr = np.frombuffer(bit_str, np.uint8)
img = np.zeros((IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH), dtype=np.uint8)
for d in range(IMG_DEPTH):
    channel = np.reshape(bit_arr[d*IMG_WIDTH*IMG_HEIGHT:(d+1)*IMG_WIDTH*IMG_HEIGHT], (IMG_HEIGHT, IMG_WIDTH))
    img[:,:,d] = channel

1.34 ms ± 69.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [50]:
f = open(PATH, 'rb')
f_size = IMG_WIDTH * IMG_HEIGHT * IMG_DEPTH

In [52]:
%%timeit
bit_str = f.read(f_size)
bit_arr = np.frombuffer(bit_str, np.uint8)
img = np.zeros((IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH), dtype=np.uint8)

for y in range(IMG_HEIGHT):
    for x in range(IMG_WIDTH):
        idx = y * IMG_WIDTH + x
        for d in range(IMG_DEPTH):
            #img[y, x, d] = np_arr[idx + d] <--- this approach does not work
            img[y, x, d] = bit_arr[idx + IMG_WIDTH * IMG_HEIGHT * d] #<--- this is how its done in the hw1 java file

562 ms ± 34.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
%%timeit
tmp_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

35.3 µs ± 4.97 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Scratch

In [24]:
print("Image shape: ", img.shape)
cv2.imshow("", cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()

Image shape:  (270, 480, 3)


In [11]:
# Read a group of frames into the buffer

f = open(PATH, 'rb')
f_size = IMG_WIDTH * IMG_HEIGHT * IMG_DEPTH

images = Buffer(60)

while True:
    bit_str = f.read(f_size)
    if not bit_str or len(bit_str) < f_size:
        break
    bit_arr = np.frombuffer(bit_str, np.uint8)
    
    img = np.zeros((IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH), dtype=np.uint8)
    for y in range(IMG_HEIGHT):
        for x in range(IMG_WIDTH):
            idx = y * IMG_WIDTH + x
            for d in range(IMG_DEPTH):
                img[y, x, d] = bit_arr[idx + IMG_WIDTH * IMG_HEIGHT * d]
                
    if len(images) == 60: break #Debug
    images.add(img)
                

In [13]:
# Display images from the buffer

for img in images.buffer:
    cv2.imshow("", cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    cv2.waitKey(33)
cv2.destroyAllWindows()

In [29]:
img[0][0]

array([248, 249, 249], dtype=uint8)

In [16]:
np_arr.shape

(388800,)

In [17]:
type(bit_str)

bytes

In [13]:
print(type(im_arr))

<class 'NoneType'>


In [9]:
?? np.frombuffer

In [7]:
len(a)

388800

In [36]:
IMG_WIDTH * IMG_HEIGHT * IMG_DEPTH

388800

In [39]:
img = np.zeros((IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH), dtype=np.uint8)

for i in range(IMG_WIDTH * IMG_HEIGHT * IMG_DEPTH):
    b = f.read(1)
    if not b:
        break
        
    x = i % IMG_WIDTH
    y = i // IMG_WIDTH
    d = i % IMG_DEPTH
    
    if y > 265: print("(x, y, d, i): ({}, {}, {}, {})".format(x, y, d, i))
    img[y, x, d] = int.from_bytes(b, sys.byteorder)&0xFF
    
    

(x, y, d, i): (0, 266, 0, 127680)
(x, y, d, i): (1, 266, 1, 127681)
(x, y, d, i): (2, 266, 2, 127682)
(x, y, d, i): (3, 266, 0, 127683)
(x, y, d, i): (4, 266, 1, 127684)
(x, y, d, i): (5, 266, 2, 127685)
(x, y, d, i): (6, 266, 0, 127686)
(x, y, d, i): (7, 266, 1, 127687)
(x, y, d, i): (8, 266, 2, 127688)
(x, y, d, i): (9, 266, 0, 127689)
(x, y, d, i): (10, 266, 1, 127690)
(x, y, d, i): (11, 266, 2, 127691)
(x, y, d, i): (12, 266, 0, 127692)
(x, y, d, i): (13, 266, 1, 127693)
(x, y, d, i): (14, 266, 2, 127694)
(x, y, d, i): (15, 266, 0, 127695)
(x, y, d, i): (16, 266, 1, 127696)
(x, y, d, i): (17, 266, 2, 127697)
(x, y, d, i): (18, 266, 0, 127698)
(x, y, d, i): (19, 266, 1, 127699)
(x, y, d, i): (20, 266, 2, 127700)
(x, y, d, i): (21, 266, 0, 127701)
(x, y, d, i): (22, 266, 1, 127702)
(x, y, d, i): (23, 266, 2, 127703)
(x, y, d, i): (24, 266, 0, 127704)
(x, y, d, i): (25, 266, 1, 127705)
(x, y, d, i): (26, 266, 2, 127706)
(x, y, d, i): (27, 266, 0, 127707)
(x, y, d, i): (28, 266, 1, 127

IndexError: index 270 is out of bounds for axis 0 with size 270

In [25]:
print(PATH)
a = cv2.imread(PATH, cv2.IMREAD_COLOR)

../data/dataset1/Videos/data_test1.rgb


In [14]:
?? cv2.imread

In [13]:
img_list = []

img = np.zeros((270, 480, 3), dtype=np.uint8)
for i in range(IMG_WIDTH * IMG_HEIGHT):
    rgb = []
    x, y = i % IMG_WIDTH, i // IMG_WIDTH
    for d in range(IMG_DEPTH):
        b = f.read(1)
        if not b: 
            break
        #rgb.append(int.from_bytes(b, sys.byteorder)&0xFF)
        img[y, x, d] = int.from_bytes(b, sys.byteorder)&0xFF
        
    #import pdb; pdb.set_trace()
    
    
    #import pdb; pdb.set_trace()

#plt.imshow(img[:, :, 2])
#cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
cv2.imshow("", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imshow("", np.zeros((270, 480, 3), dtype=np.uint8) + 255)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
?? cv2.Mat

In [19]:
?? cv2.imshow

In [13]:
n_lines = 0

while True:
    line = f.readline()
    if not line:
        break
    n_lines += 1

In [14]:
n_lines

15961173

In [15]:
480*270

129600

In [ ]:
byte = f.read(1)
while(byte)